In [ ]:
from pathlib import Path

import kagglehub
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from xgboost import XGBRegressor, XGBRFRegressor


Download the supplementary data from Kaggle for artist info

In [25]:
if Path("data/spotify_artist_data.csv").exists():
    artist_stats = pd.read_csv("data/spotify_artist_data.csv")
else:
    path = kagglehub.dataset_download("adnananam/spotify-artist-stats")
    artist_stats = pd.read_csv(path + "/spotify_artist_data.csv", index_col=0)

    # Remove error rows b/c the creator didn't process correctly
    artist_stats = artist_stats[artist_stats["Lead Streams"] != "Lead Streams"]

    # Cast numeric columns to int
    for col in ["Lead Streams", "Feats", "Tracks", "One Billion", "100 Million"]:
        artist_stats[col] = artist_stats[col].str.replace(",", "").astype(int)

    # Remove the last updated column, it's not useful/relevant
    artist_stats = artist_stats.drop(columns=["Last Updated"])

    artist_stats.to_csv("data/spotify_artist_data.csv", index=False)

artist_stats.head()

,Artist Name,Lead Streams,Feats,Tracks,One Billion,100 Million
0,Drake,50162292808,19246513666,262,6,130
1,Bad Bunny,44369032140,5391990975,163,5,118
2,Ed Sheeran,38153682361,2791278201,240,10,62
3,The Weeknd,34767779741,4288903657,186,8,72
4,Taylor Swift,32596728109,424053296,323,1,96


Download the dataset from HuggingFace using Pandas, and drop the extra index column. The `na`/`NaN` values were dropped from the `artists` column because that column is used to merge the supplementary data above with the main dataset.

In [26]:
# Pulled dataset from HF, dropped unneeded index column
if Path("data/spotify_tracks.csv").exists():
    df = pd.read_csv("data/spotify_tracks.csv")
else:
    df = (
        pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")
        .drop("Unnamed: 0", axis=1)
        .dropna(subset=["artists"])
    )

    df["duration_s"] = df["duration_ms"] / 1000
    df = df.drop(columns=["duration_ms"])  # Drop original duration column, keep seconds

    df.to_csv("data/spotify_tracks.csv", index=False)

df_nodupe = df.drop_duplicates(subset=["track_id"]).copy()

df.head()

,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_s
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,230.666
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,149.610
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,210.826
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,201.933
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,198.853


Adding in more information to the main dataset using each artist's stats. If there are two or more artists present, the stats are averaged.

In [29]:
if not Path("data/spotify_tracks_processed.csv").exists():
    art_stats_name = set(artist_stats["Artist Name"].values)
    lead_streams, feats, tracks, one_billion, hundred_million = [], [], [], [], []

    for row in tqdm(df_nodupe.iterrows(), total=df_nodupe.shape[0], desc="Processing rows"):
        artists = [x.strip() for x in row[1]["artists"].split(";")]
        temp_lead_streams, temp_feats, temp_tracks, temp_one_billion, temp_hundred_million = (
            [],
            [],
            [],
            [],
            [],
        )

        for artist in artists:
            if artist in art_stats_name:
                temp_lead_streams.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["Lead Streams"].values[0]
                )
                temp_feats.append(artist_stats[artist_stats["Artist Name"] == artist]["Feats"].values[0])
                temp_tracks.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["Tracks"].values[0]
                )
                temp_one_billion.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["One Billion"].values[0]
                )
                temp_hundred_million.append(
                    artist_stats[artist_stats["Artist Name"] == artist]["100 Million"].values[0]
                )

        for col, temp in zip(
            [lead_streams, feats, tracks, one_billion, hundred_million],
            [temp_lead_streams, temp_feats, temp_tracks, temp_one_billion, temp_hundred_million],
            strict=True,
        ):
            if len(temp) > 0:
                col.append(np.mean(temp))
            else:
                col.append(0)

    df_nodupe["lead_streams"] = lead_streams
    df_nodupe["feats"] = feats
    df_nodupe["tracks"] = tracks
    df_nodupe["one_billion"] = one_billion
    df_nodupe["hundred_million"] = hundred_million

    g_dummy = pd.get_dummies(df["track_genre"]).groupby(df["track_id"]).sum().astype(int).reset_index()

    dummy_val = g_dummy.copy()
    dummy_val["total"] = dummy_val.sum(axis=1, numeric_only=True)
    dummy_val = dummy_val[["track_id", "total"]].sort_values("track_id", ascending=True)

    process_check = (
        df.groupby("track_id")
        .size()
        .to_frame("total")
        .reset_index()
        .sort_values("track_id", ascending=True)
    )

    for df1, df2 in zip(process_check.iterrows(), dummy_val.iterrows(), strict=True):
        assert (df1[1]["total"] == df2[1]["total"]) and (df1[1]["track_id"] == df2[1]["track_id"])

    df = df_nodupe.merge(g_dummy, on="track_id").drop(
        ["track_id", "artists", "album_name", "track_name", "track_genre"], axis=1
    )
    df["explicit"] = df["explicit"].astype(int)
    df.to_csv("data/spotify_tracks_processed.csv", index=False)

else:
    df = pd.read_csv("data/spotify_tracks_processed.csv")

In [30]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the RandomForestRegressor model
model = LinearRegression(
    n_jobs=-1,
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output the results
print(f"R²: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R²: 0.34835154909974264
MSE: 274.2463457682768
RMSE: 16.56038483152722
MAE: 11.862547512613704


In [31]:
df.describe()

,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,spanish,study,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music
count,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,...,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000
mean,33.198808,0.085848,0.562166,0.634458,5.283530,-8.498994,0.636973,0.087442,0.328285,0.173415,...,0.011143,0.011143,0.011143,0.011143,0.011143,0.011143,0.011143,0.011143,0.011143,0.011143
std,20.580640,0.280141,0.176692,0.256606,3.559912,5.221518,0.480875,0.113278,0.338321,0.323849,...,0.104973,0.105185,0.104973,0.104973,0.104973,0.104973,0.105079,0.105291,0.105079,0.105079
min,0.000000,0.000000,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.450000,0.457000,2.000000,-10.322250,0.000000,0.036000,0.017100,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,0.000000,0.576000,0.676000,5.000000,-7.185000,1.000000,0.048900,0.188000,0.000058,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,0.000000,0.692000,0.853000,8.000000,-5.108000,1.000000,0.085900,0.625000,0.097625,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,100.000000,1.000000,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,...,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000


In [32]:
df.corr()["popularity"].sort_values(ascending=False)

popularity          1.000000
pop-film            0.134407
k-pop               0.122339
hundred_million     0.106079
chill               0.105386
                      ...   
detroit-techno     -0.113376
latin              -0.127165
instrumentalness   -0.127477
romance            -0.141027
iranian            -0.157936
Name: popularity, Length: 134, dtype: float64

### Backfill missing `lead_streams` values

In [33]:
# Create mask for rows where lead_streams is 0
mask = df['lead_streams'] == 0

# Split data into features (X) and target (y)
X_train = df[~mask].drop(['lead_streams', 'popularity'], axis=1)
y_train = df[~mask]['lead_streams']

# Prepare features for prediction
X_pred = df[mask].drop(['lead_streams', 'popularity'], axis=1)

# Initialize and train the RandomForestRegressor
rf_model = RandomForestRegressor(
    n_estimators=200, 
    random_state=42, 
    n_jobs=-1, 
    max_features='sqrt',
    verbose=1
)
rf_model.fit(X_train, y_train)

# Make predictions for empty values
predictions = rf_model.predict(X_pred)

# Fill in the empty values
df.loc[mask, 'lead_streams'] = predictions

# Verify no more zeros in lead_streams
print(f"Number of zeros in lead_streams: {(df['lead_streams'] == 0).sum()}")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.7s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s


Number of zeros in lead_streams: 0


[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.2s finished


### Add clusters as additional features

In [ ]:
std_df = StandardScaler().fit_transform(df[df.columns.difference(["popularity"])])
kmeans = KMeans(n_clusters=40, random_state=42)
kmeans.fit(std_df)
df["cluster"] = kmeans.labels_
df["cluster"] = df["cluster"].astype("category")

df

,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,study,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music,cluster
0,73,0,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,...,0,0,0,0,0,0,0,0,0,12
1,55,0,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,...,0,0,0,0,0,0,0,0,0,16
2,57,0,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,...,0,0,0,0,0,0,0,0,0,12
3,71,0,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,...,0,0,0,0,0,0,0,0,0,12
4,82,0,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,...,0,0,0,0,0,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89735,21,0,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,...,0,0,0,0,0,0,0,0,1,12
89736,22,0,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,...,0,0,0,0,0,0,0,0,1,19
89737,22,0,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,...,0,0,0,0,0,0,0,0,1,12
89738,41,0,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,...,0,0,0,0,0,0,0,0,1,12


### Manual correlation check

In [35]:
dfc = df.corr()

# Create mask for correlations > abs(0.50)
mask = np.abs(dfc) > 0.50

# Get upper triangle of mask to avoid duplicates
mask_upper = np.triu(mask, k=1)

# Find correlation pairs exceeding threshold
high_corr = []
for i in range(len(dfc.columns)):
    for j in range(i + 1, len(dfc.columns)):
        if mask_upper[i, j]:
            high_corr.append({"var1": dfc.columns[i], "var2": dfc.columns[j], "corr": dfc.iloc[i, j]})

# Convert to dataframe and sort by absolute correlation
high_corr_df = pd.DataFrame(high_corr)
high_corr_df = high_corr_df.sort_values("corr", key=abs, ascending=False)

print("Correlations > |0.50|:")
print(high_corr_df.to_string(index=False))

Correlations > |0.50|:
             var1             var2      corr
singer-songwriter       songwriter  1.000000
     lead_streams  hundred_million  0.952045
     lead_streams      one_billion  0.822557
           reggae        reggaeton  0.801791
           energy         loudness  0.758774
           latino        reggaeton  0.736928
           energy     acousticness -0.732569
              dub          dubstep  0.723472
      one_billion  hundred_million  0.706854
             punk        punk-rock  0.624188
      speechiness           comedy  0.623655
              edm            house  0.619816
           latino           reggae  0.614418
     lead_streams featured_streams  0.612210
 featured_streams  hundred_million  0.593427
            latin           latino  0.590402
         alt-rock      alternative  0.588235
  featured_tracks        classical  0.583836
         loudness     acousticness -0.582664
            indie        indie-pop  0.573530
 featured_streams      one_billi

### Drop highly correlated columns

- `singer-songwriter`
  - Removed since it is an identical match to `songwriter`
- `hundred_million`, `one_billion`, and `featured_streams`
  - High correlation to `lead_streams`, which was from the same dataset and all columns were used during backfilling

In [36]:
df = df.drop(columns=["singer-songwriter", "hundred_million", "one_billion", "featured_streams"])

In [37]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the RandomForestRegressor model
model = LinearRegression(
    n_jobs=-1,
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output the results
print(f"R²: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R²: 0.3474713146844247
MSE: 274.616792550567
RMSE: 16.571565784516775
MAE: 11.881378178125107


In [38]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestRegressor model
model = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1, max_features="sqrt", bootstrap=True
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output the results
print(f"R²: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R²: 0.5559460661296558
MSE: 185.71911270467845
RMSE: 13.627879978363415
MAE: 9.219101136800697


In [39]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=40, random_state=42)

# Initialize the XGBRegressor model
model = XGBRegressor(
    tree_method="hist",
    n_estimators=300,
    n_jobs=-1,
    random_state=42,
    enable_categorical=True,
    early_stopping_rounds=50,
)

# Fit the model to the training data
model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Make predictions on the test set
y_pred = model.predict(X_test)

[0]	validation_0-rmse:16.98124
[1]	validation_0-rmse:16.20884
[2]	validation_0-rmse:15.90300
[3]	validation_0-rmse:15.47981
[4]	validation_0-rmse:15.36346
[5]	validation_0-rmse:15.16049
[6]	validation_0-rmse:15.16032
[7]	validation_0-rmse:15.09215
[8]	validation_0-rmse:14.69200
[9]	validation_0-rmse:14.52064
[10]	validation_0-rmse:14.52222
[11]	validation_0-rmse:14.48463
[12]	validation_0-rmse:14.45850
[13]	validation_0-rmse:14.34518
[14]	validation_0-rmse:14.19655
[15]	validation_0-rmse:14.18575
[16]	validation_0-rmse:14.11949
[17]	validation_0-rmse:14.10056
[18]	validation_0-rmse:13.97183
[19]	validation_0-rmse:13.93620
[20]	validation_0-rmse:13.94521
[21]	validation_0-rmse:13.90971
[22]	validation_0-rmse:13.86521
[23]	validation_0-rmse:13.83083
[24]	validation_0-rmse:13.80697
[25]	validation_0-rmse:13.70662
[26]	validation_0-rmse:13.70409
[27]	validation_0-rmse:13.68022
[28]	validation_0-rmse:13.68781
[29]	validation_0-rmse:13.31178
[30]	validation_0-rmse:13.27621
[31]	validation_0-

In [40]:
top_n = 50

# Get feature importances and column names
feature_importances = pd.DataFrame(
    {"feature": X_train.columns, "importance": model.feature_importances_}
)

# Sort by importance and get top_n
top_features = feature_importances.sort_values("importance", ascending=False).head(top_n)

# Display results
print(f"Top {top_n} most important features:")
print(top_features.to_string(index=False))

Top 50 most important features:
        feature  importance
        romance    0.099651
        iranian    0.056321
      metalcore    0.044727
          latin    0.043832
           kids    0.040565
          chill    0.036557
     honky-tonk    0.035475
      breakbeat    0.025325
        j-dance    0.024959
            sad    0.023523
         techno    0.022933
          anime    0.022562
          party    0.019671
          tango    0.019620
          k-pop    0.018426
       pop-film    0.018394
       afrobeat    0.018330
      grindcore    0.016354
           soul    0.015925
 minimal-techno    0.015803
      classical    0.013716
            idm    0.011697
          piano    0.011697
    heavy-metal    0.010777
          forro    0.010414
        hip-hop    0.009985
    alternative    0.009750
        cluster    0.009698
         latino    0.009627
 detroit-techno    0.009430
  chicago-house    0.009295
           rock    0.008702
        ambient    0.008640
          opera 

In [41]:
# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output the results
print(f"R²: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R²: 0.5129150152206421
MSE: 204.3378448486328
RMSE: 14.294678688049316
MAE: 9.804095268249512


In [42]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[df.columns.difference(["popularity"])]
y = df["popularity"]

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the XGBRFRegressor model
model = XGBRFRegressor(
    tree_method="hist", n_estimators=200, n_jobs=-1, random_state=42, enable_categorical=True
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output the results
print(f"R²: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R²: 0.23506581783294678
MSE: 321.92266845703125
RMSE: 17.942203521728516
MAE: 14.01779556274414


In [43]:
top_n = 50

# Get feature importances and column names
feature_importances = pd.DataFrame(
    {"feature": X_train.columns, "importance": model.feature_importances_}
)

# Sort by importance and get top_n
top_features = feature_importances.sort_values("importance", ascending=False).head(top_n)

# Display results
print(f"Top {top_n} most important features:")
print(top_features.to_string(index=False))

Top 50 most important features:
        feature  importance
        romance    0.116647
          latin    0.089084
        cluster    0.082707
        iranian    0.055935
           kids    0.053665
     honky-tonk    0.047802
      metalcore    0.045511
       pop-film    0.042793
    alternative    0.035005
         techno    0.034517
          dance    0.034318
          chill    0.033964
          opera    0.030056
       afrobeat    0.029712
           rock    0.026533
   lead_streams    0.021848
           funk    0.020846
 minimal-techno    0.019999
      classical    0.018851
 detroit-techno    0.011958
      grindcore    0.011233
            pop    0.010132
         reggae    0.009972
  chicago-house    0.009672
           soul    0.008708
            idm    0.008527
         disney    0.006789
          indie    0.006618
       children    0.006004
        hip-hop    0.005899
featured_tracks    0.004757
        electro    0.004564
        turkish    0.004553
         j-rock 

In [44]:
# Assuming 'df' is your DataFrame and 'features' is a list of feature column names
X = df[top_features["feature"].to_list()]
y = df["popularity"]

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize the RandomForestRegressor model
model = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1, max_features="sqrt", bootstrap=True
)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output the results
print(f"R²: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R²: 0.49929462026689475
MSE: 210.7219322145515
RMSE: 14.516264402887938
MAE: 10.169323500016093


Can skip stacked model maybe, its performance is only slightly above the RFRs.

```md
Stacked Model R^2: 0.537192088407299
Stacked Model MSE: 194.77277721078033
Stacked Model RMSE: 13.956101791359231
Stacked Model MAE: 9.468840211962455
```

In [45]:
# # Splitting the data
# X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(
#     df[top_features["feature"].to_list()], df["popularity"], test_size=0.3, random_state=42
# )

# # Base models
# base_models = [
#     ("rf", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
#     ("gb", GradientBoostingRegressor(n_estimators=200, random_state=42)),
#     ("lr", LinearRegression()),
# ]

# # Meta-model (Level 2 model)
# meta_model = LinearRegression()

# # Stacking model
# stacked_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)
# stacked_model.fit(X_train_scaled, y_train)

# # Predictions and MSE
# y_pred_stacked = stacked_model.predict(X_test_scaled)
# mse_stacked = mean_squared_error(y_test, y_pred_stacked)
# print(f"Stacked Model MSE: {mse_stacked}")


In [46]:
# r2 = r2_score(y_test, y_pred_stacked)
# mse = mean_squared_error(y_test, y_pred_stacked)
# rmse = root_mean_squared_error(y_test, y_pred_stacked)
# mae = mean_absolute_error(y_test, y_pred_stacked)

# print(f"Stacked Model R^2: {r2}")
# print(f"Stacked Model MSE: {mse}")
# print(f"Stacked Model RMSE: {rmse}")
# print(f"Stacked Model MAE: {mae}")
